In [ ]:
import numpy as np 
import pandas as pd
import os 
import sys
import shutil
import pathlib
import fastai.vision.all as faiv
import gc
import torch

sys.path.append('../input/pytorch-image-models')
pathlib.Path('/root/.cache/torch/hub/checkpoints/').mkdir(parents=True, exist_ok=True) 
shutil.copy('../input/swim-package/swin_large_patch4_window7_224_22kto1k.pth', '/root/.cache/torch/hub/checkpoints/swin_large_patch4_window7_224_22kto1k.pth')

from timm import create_model


In [ ]:
def getKFoldIndices(inp_df, k=5):
    """
    Return k columns of booleans such that each entry in the original DataFrame corresponds to a row with a unique "1" entry in the k columns. 
    The columns correspond to training set splits to be used in k-fold cross validation.
    """
    
    no_dps = inp_df.shape[0]
    folds = np.zeros([k, no_dps])
    indices = np.linspace(1, no_dps, no_dps)
    np.random.shuffle(indices)
    fold_indices = np.array_split(indices, k)
    indices = np.sort(indices)
    for i in range(k):
        folds[i] = np.in1d(indices, fold_indices[i])
        
    return folds.T

In [ ]:
train_im_dir = '../input/petfinder-pawpularity-score/train'
train_df = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/train.csv') 

# fastai reads image directories directly from pandas tables, so we modify the image ID to correspond to its path in the directory.
train_df['Id'] = train_df['Id'].apply(lambda x: f"../input/petfinder-pawpularity-score/train/{x}.jpg")

# Divide training data target to be values between 0-1 in order to treat pawpularity prediction as a binary classification problem.
train_df['Pawpularity'] = train_df['Pawpularity']/100

# Get indices corresponding to five cv splits
folds = getKFoldIndices(train_df, k=5)


# Fast AI accepts columns of booleans as an input for CV folds. So, we create these columns 
# whose entries are 1 or 0 dependening on whether the data entry should be include in the
# validation or training set of a CV fold, respectively.

folds_df = pd.DataFrame(folds)
folds_df.columns = ['Fold_1', 'Fold_2', 'Fold_3', 'Fold_4', 'Fold_5']
train_df = pd.concat([train_df, folds_df], axis=1)
train_df.head()

In [ ]:
def getFoldModel(train_df, fold_str, lr = 5.6e-5):
    """Function that exports the parameters of a trained SWIN model for a given CV fold."""
    
    # Get training set batches from the training dataframe. Note that the valid_col variable corresponds to the current CV fold.
    fastai_dl = faiv.ImageDataLoaders.from_df(df = train_df,
                                     seed = 1997,
                                     fn_col = 'Id',
                                     label_col = 'Pawpularity',
                                     valid_col = fold_str,
                                     y_block = faiv.RegressionBlock,
                                     item_tfms = faiv.Resize(224),
                                     bs = 16)
    
    # Get pretrained SWIN model. The 224x224x3 model is used as there is not enough GPU memory to train the 384x384x3 SWIN model.
    model = create_model('swin_large_patch4_window7_224', pretrained = True, num_classes = 1)
    
    # Update the SWIN parameters on the training set using 
    fai_learner = faiv.Learner(fastai_dl, model, loss_func=faiv.BCEWithLogitsLossFlat())
    fai_learner.fit_one_cycle(10, lr, cbs=[faiv.SaveModelCallback(), faiv.EarlyStoppingCallback(patience = 2)])
    fai_learner.recorder.plot_loss()
    out_str = f'SWIN_v2_{fold_str}.pkl'
    fai_learner.export(out_str)
    return fai_learner

In [ ]:
learners = []
for fold_index,fold_str in enumerate(['Fold_1', 'Fold_2', 'Fold_3', 'Fold_4', 'Fold_5']):
    print(fold_str)
    getFoldModel(train_df, fold_str)
    
    # Collect garbage and empty GPU cache to free up memory.
    torch.cuda.empty_cache()
    gc.collect()